In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [20]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "citibike_tabular_data_28d.parquet")
df.head()



,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 11:00:00,5329.03


In [21]:
df_est = df.copy()
df_est["pickup_hour"] = pd.to_datetime(df_est["pickup_hour"])
df_est["pickup_hour"] = df_est["pickup_hour"].dt.tz_localize("UTC").dt.tz_convert("America/New_York")


In [22]:
df.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 11:00:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 11:00:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 11:00:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 11:00:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 11:00:00,5329.03


In [23]:
df_est.head()

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,target,pickup_hour,pickup_location_id
0,1,0,0,0,0,0,0,0,0,0,...,0,0,5,12,15,12,4,6,2023-01-25 06:00:00-05:00,5329.03
1,0,0,0,0,0,0,0,0,0,0,...,0,1,5,12,21,6,8,8,2023-01-26 06:00:00-05:00,5329.03
2,0,0,0,0,0,0,0,0,0,0,...,0,1,1,10,12,6,8,3,2023-01-27 06:00:00-05:00,5329.03
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,6,5,10,23,2023-01-28 06:00:00-05:00,5329.03
4,15,12,10,24,23,14,12,6,5,5,...,0,0,0,1,2,9,6,14,2023-01-29 06:00:00-05:00,5329.03


In [24]:
from datetime import datetime
from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2023, 9, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(657, 674)
(657,)
(366, 674)
(366,)


In [25]:
import numpy as np

class BaselineModelPreviousHour:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]


In [26]:
model = BaselineModelPreviousHour()
predictions = model.predict(X_test)


In [27]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


5.7459


In [40]:
import os
import mlflow

mlflow.set_tracking_uri("https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow")
os.environ["MLFLOW_TRACKING_USERNAME"] = "ryallavinuthnareddy"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "92dd14cdf34bef08d8871bf22dbdc0869a9198cf"
# ==== ✅ Step 2: Confirm that it picked up your credentials ====
print("Tracking URI:", mlflow.get_tracking_uri())
print("MLflow username:", os.getenv("MLFLOW_TRACKING_USERNAME"))

Tracking URI: https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow
MLflow username: ryallavinuthnareddy


In [41]:
log_model_to_mlflow(
    model,
    X_test,
    "BaselineModelPreviousHour",
    "mean_absolute_error",
    score=test_mae
)


INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousHour
INFO:src.experiment_utils:Logged mean_absolute_error: 5.745901639344262
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.
Registered mo

🏃 View run lyrical-fawn-200 at: https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow/#/experiments/0/runs/4da996d35c0d416e8c5b913a56569860
🧪 View experiment at: https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow/#/experiments/0


In [42]:
class BaselineModelPreviousWeek:
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*24}"]  # rides_t-168


In [43]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)


In [44]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

9.7951


In [45]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeek", "mean_absolute_error", score=test_mae)

2025/05/10 22:36:58 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelPreviousWeek' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelPreviousWeek
INFO:src.experiment_utils:Logged mean_absolute_error: 9.795081967213115
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#

🏃 View run bouncy-gnu-33 at: https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow/#/experiments/1/runs/3bc874d8bdc34efd80a9676fe816ffe3
🧪 View experiment at: https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow/#/experiments/1


In [46]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


9.7951


In [47]:
import numpy as np
import pandas as pd

class BaselineModelLast4Weeks:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass  # No training needed for baseline

    def predict(self, X_test: pd.DataFrame) -> np.array:
        last_4_weeks_columns = [
            f"rides_t-{7*24}",   # 1 week ago
            f"rides_t-{14*24}",  # 2 weeks ago
            f"rides_t-{21*24}",  # 3 weeks ago
            f"rides_t-{28*24}"   # 4 weeks ago
        ]
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")
        return X_test[last_4_weeks_columns].mean(axis=1).to_numpy()


In [48]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)


In [49]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


8.2787


In [50]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

8.2787


In [51]:
log_model_to_mlflow(model, X_test, "BaselineModelLast4Weeks", "mean_absolute_error", score=test_mae)


2025/05/10 22:42:01 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelLast4Weeks' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: BaselineModelLast4Weeks
INFO:src.experiment_utils:Logged mean_absolute_error: 8.278688524590164
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#hand

🏃 View run unequaled-mouse-219 at: https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow/#/experiments/2/runs/6cbadda303e0422a855552c843727b6c
🧪 View experiment at: https://dagshub.com/ryallavinuthnareddy/citibikeproject.mlflow/#/experiments/2


In [52]:
X_test.head()  # Get the total number of rows


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
219,14,16,19,19,25,46,56,48,29,23,...,0,0,0,4,15,14,13,16,2023-09-01 11:00:00,5329.03
220,45,43,34,49,51,63,63,49,52,29,...,0,0,0,2,7,4,12,32,2023-09-02 11:00:00,5329.03
221,26,32,36,46,62,50,58,48,47,25,...,1,2,0,1,1,12,14,12,2023-09-03 11:00:00,5329.03
222,9,8,19,14,16,24,49,76,49,35,...,0,1,0,1,5,5,17,21,2023-09-04 11:00:00,5329.03
223,8,13,23,15,15,27,66,89,80,50,...,0,0,0,0,0,20,17,12,2023-09-05 11:00:00,5329.03


In [53]:
from src.plot_utils import plot_aggregated_time_series, plot_prediction
plot_aggregated_time_series, plot_prediction

(<function src.plot_utils.plot_aggregated_time_series(features: pandas.core.frame.DataFrame, targets: pandas.core.series.Series, row_id: int, predictions: Optional[pandas.core.series.Series] = None)>,
 <function src.plot_utils.plot_prediction(features: pandas.core.frame.DataFrame, prediction: int)>)